In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# importar todos os pacotes necessários para usar os vários algoritmos de classificação
from sklearn.linear_model import LogisticRegression  # for Logistic Regression algorithm
from sklearn.model_selection import train_test_split #to split the dataset for training and testing
from sklearn.neighbors import KNeighborsClassifier  # for K nearest neighbours
from sklearn import svm  #for Support Vector Machine (SVM) Algorithm
from sklearn import metrics #for checking the model accuracy
from sklearn.tree import DecisionTreeClassifier #for using Decision Tree Algoithm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
roupas = pd.read_csv("../input/summer-products-and-sales-in-ecommerce-wish/summer-products-with-rating-and-performance_2020-08.csv")  #carregando o conjunto de dados

In [ ]:
roupas.head(3) #mostra as 3 primeiras linhas do conjunto de dados

In [ ]:
roupas.info()

In [ ]:
#Dropando todas as informações não importantes
roupas.drop('currency_buyer',axis=1,inplace=True)
roupas.drop('has_urgency_banner',axis=1,inplace=True)
roupas.drop('urgency_text',axis=1,inplace=True)
roupas.drop('crawl_month',axis=1,inplace=True)
roupas.drop('merchant_name',axis=1,inplace=True)
roupas.drop('merchant_title',axis=1,inplace=True)
roupas.drop('merchant_has_profile_picture',axis=1,inplace=True)
roupas.drop('merchant_profile_picture',axis=1,inplace=True)
roupas.drop('merchant_rating_count',axis=1,inplace=True)
roupas.drop('merchant_info_subtitle',axis=1,inplace=True)
roupas.drop('merchant_id',axis=1,inplace=True)
roupas.drop('uses_ad_boosts',axis=1,inplace=True)
roupas.drop('badges_count',axis=1,inplace=True)
roupas.drop('badge_local_product',axis=1,inplace=True)
roupas.drop('badge_fast_shipping',axis=1,inplace=True)
roupas.drop('badge_product_quality',axis=1,inplace=True)
roupas.drop('shipping_is_express',axis=1,inplace=True)
roupas.drop('shipping_option_name',axis=1,inplace=True)
roupas.drop('shipping_option_price',axis=1,inplace=True)
roupas.drop('inventory_total',axis=1,inplace=True)
roupas.drop('product_url',axis=1,inplace=True)
roupas.drop('product_picture',axis=1,inplace=True)
roupas.drop('product_id',axis=1,inplace=True)
roupas.drop('theme',axis=1,inplace=True)

roupas = roupas.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False) #remove rows com dados nulos

In [ ]:
roupas.info()

In [ ]:
roupas[roupas.price==11].head(3)

In [ ]:
roupas.shape

In [ ]:
fig = roupas[roupas.product_color=='black'].plot(kind='scatter',x='rating',y='price',color='black', label='Cor Preta')
roupas[roupas.product_color=='green'].plot(kind='scatter',x='rating',y='price',color='lime', label='Cor Verde',ax=fig)
roupas[roupas.product_color=='red'].plot(kind='scatter',x='rating',y='price',color='red', label='Cor Verde',ax=fig)
fig.set_xlabel("Avaliação do Produto")
fig.set_ylabel("Preço")
fig.set_title("Avaliação do Produto VS Preço para cada cor")
fig=plt.gcf() # retorna a figura que está em construção
fig.set_size_inches(12,8)
plt.show()

In [ ]:
plt.close()
roupas.hist(edgecolor='white', linewidth=1.2)
fig=plt.gcf()
fig.set_size_inches(20,16)
plt.show()

In [ ]:
plt.figure(figsize=(20,12)) 
sns.heatmap(roupas.corr(),annot=True,cmap='cubehelix_r') #draws  heatmap with input as the correlation matrix calculted by(iris.corr())
plt.show()

# Recomendação - Filtro Demográfico

In [ ]:
C = roupas['rating'].mean()
C

In [ ]:
m = roupas['rating_count'].quantile(0.8)
m

In [ ]:
roupas = roupas.copy().loc[roupas['rating_count'] >= m]
roupas.shape

In [ ]:
def weighted_rating(x, m=m, C=C):
    v = x['rating_count']
    R = x['rating']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

roupas['nota'] = roupas.apply(weighted_rating, axis=1)

#Organizando lista baseada na nota das roupas
roupas = roupas.sort_values('nota', ascending=False)

#Top 10 roupas
roupas[['title_orig', 'price', 'rating', 'nota']].head(10)

Recomendaçao - Unidades vendidas

In [ ]:
roupas = roupas.sort_values('units_sold', ascending=False)

roupas[['title_orig', 'price', 'units_sold', 'nota']].head(10)

Recomendação - Rating

In [ ]:
roupas = roupas.sort_values('rating', ascending=False)

roupas[['title_orig', 'price', 'rating', 'nota']].head(10)

# Recomendação -  Filtragem baseada em conteudo

In [ ]:
tfidf = TfidfVectorizer(stop_words='english')

roupas['title_orig'] = roupas['title_orig'].fillna('')

tfidf_matrix = tfidf.fit_transform(roupas['title_orig'])

tfidf_matrix.shape

In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
indices = pd.Series(roupas.index, index=roupas['title_orig']).drop_duplicates()
indices.head()

In [ ]:
# Obtenha o índice da roupa que corresponde ao título
idx = indices['Top Swimsuit Sets Fashion Swimwear Women Bikini 2016 Sexy Padded']
idx

In [ ]:
# Obtenha as pontuações de semelhança entre pares de todas as roupas com aquela roupa
sim_scores = list(enumerate(cosine_sim[idx]))
#type(sim_scores)
#print(sim_scores)

# Classifique as roupas com base nas pontuações de similaridade
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#print(sim_scores)

In [ ]:
# Obtenha as pontuações das 10 roupas mais semelhantes
sim_scores = sim_scores[1:11]
sim_scores

In [ ]:
# Obtenha os índices de roupas
movie_indices = [i[0] for i in sim_scores]
movie_indices

In [ ]:
roupas['title_orig'].iloc[movie_indices]

SIMPLIFICANDO!!!

In [ ]:
# Função que recebe o título do filme como entrada e produz a maioria dos filmes semelhantes
def get_recommendations(title_orig, cosine_sim=cosine_sim):
    # Obtenha o índice do filme que corresponde ao título
    idx = indices[title_orig]
    
        # Obtenha as pontuações de semelhança entre pares de todas as roupas com aquela roupa
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Classifique as roupas com base nas pontuações de similaridade
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtenha as pontuações das 10 roupas mais semelhantes
    sim_scores = sim_scores[1:11]

    # Obtenha os índices das roupas
    movie_indices = [i[0] for i in sim_scores]

    # Retorne os 10 principais roupas mais semelhantes
    return roupas['title_orig'].iloc[movie_indices]


In [ ]:
get_recommendations("Pnma Men Summer Quick Dry Shorts Mens Swim Beach Shorts Swim Shorts Beach Wear Sports Gym")

# Classificação

In [ ]:
train, test = train_test_split(roupas, test_size = 0.1)# neste, nossos dados principais são divididos em treinar e testar
# o atributo test_size = 0.2 divide os dados em uma proporção de 90% e 10%. treinar = 90% e teste = 10%
print(train.shape)
print(test.shape)

In [ ]:
train

In [ ]:
train_X = train[['price','rating_count','rating','units_sold']]# taking the training data features
train_y=train.units_sold # output of our training data


test_X= test[['price','rating_count','rating','units_sold']] # taking test data features
test_y =test.units_sold  # output value of test data

In [ ]:
test_X.head(2)

Support Vector Machine (SVM)

In [ ]:
model = svm.SVC() #selecione o algoritmo
model.fit(train_X,train_y) # treinamos o algoritmo com os dados de treinamento e a saída do treinamento



prediction=model.predict(test_X) # agora passamos os dados de teste para o algoritmo treinado
print('The accuracy of the SVM is:',metrics.classification_report(test_y, prediction)) 
#print('The precision of the SVM is:',metrics.precision_score(prediction,test_y, average='macro'))
#print('The Recall of the SVM is:',metrics.recall_score(prediction,test_y, average='macro')) 
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html#sklearn.metrics.precision_score
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html#sklearn.metrics.recall_score
#passamos a saída prevista pelo modelo e a saída real

In [ ]:
model=KNeighborsClassifier(n_neighbors=3) #this examines 3 neighbours for putting the new data into a class
model.fit(train_X,train_y)
prediction=model.predict(test_X)
print('The accuracy of the KNN is',metrics.accuracy_score(test_y, prediction))